# 金融风控相关业务之风控报表学习目标
- 知道信贷业务是如何运行的
- 信贷业务行为路径转化为可视化漏斗


# 小结：
- 拉宽表
    - 数据可能是在不同的表保存，取数据时候是不是需要考虑一些边界的问题
    - 考虑如何去JOIN那张是朱标，如何JOIN不会丢信息
- 在宽表的基础上计算指标：加减乘除在算之前，每个指标的计算方法一定要搞清楚
- 整理结果

In [1]:
%%sql
# 创建用户表
CREATE TABLE `u_user` (
  `id` bigint NOT NULL AUTO_INCREMENT COMMENT '自增主键',
  `username` varchar(64) ,
  `mobile` varchar(20) ,
  `password` varchar(64) ,
  `nickname` varchar(64) ,
  `role_type` int NOT NULL DEFAULT '0' COMMENT '角色(-1:普通用户)',
  `type` int NOT NULL DEFAULT '0' COMMENT '(暂时保留)0：借款用户,1:资金账户用户',
  `status` int NOT NULL DEFAULT '0' COMMENT '用户状态',
  `on_off` char(4) NOT NULL DEFAULT '1' COMMENT '开关(0:关闭,1:开启)',
  `inserttime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT '插入时间',
  `updatetime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP COMMENT '更新时间',
  `isactive` tinyint(1) NOT NULL DEFAULT '1' COMMENT '逻辑删除(1,有效)',
  PRIMARY KEY (`id`) USING BTREE,
  KEY `idx_mobile` (`mobile`,`on_off`,`status`,`isactive`,`password`) USING BTREE
) ENGINE=InnoDB AUTO_INCREMENT=11050 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci COMMENT='用户表';


Exception: [42S01][1050] Table 'u_user' already exists

In [2]:
%%sql
# 创建用户信息表
CREATE TABLE `u_personal_info` (
  `id` bigint NOT NULL AUTO_INCREMENT COMMENT '自增主键',
  `user_id` bigint NOT NULL DEFAULT '0' COMMENT '用户id',
  `loan_purpose` varchar(255)  COMMENT '借款目的，tb_data_dict表中type=4',
  `sex` varchar(20)  COMMENT '性别(0：男 1：女)，tb_data_dict表中type=1',
  `birthdate` varchar(20)  COMMENT '出生日期',
  `birthplace` varchar(256)  COMMENT '出生地',
  `religion` varchar(20)  COMMENT '宗教，tb_data_dict表中type=3',
  `education` varchar(20)  COMMENT '教育程度，tb_data_dict表中type=2',
  `nation` varchar(20)  COMMENT '民族',
  `tribe` varchar(20)  COMMENT '部落，tb_data_dict表中type=10',
  `living_state` varchar(20)  COMMENT '居住状态，tb_data_dict表中type=9',
  `province` varchar(128)  COMMENT '居住省',
  `city` varchar(128)  COMMENT '居住市',
  `district` varchar(128)  COMMENT '居住区',
  `address` varchar(500)  COMMENT '居住详细地址',
  `children_number` varchar(20)  COMMENT '孩子数量(字典.type=15)',
  `number_of_provide` int DEFAULT NULL COMMENT '需供养人数',
  `phone_use_duration` varchar(20)  COMMENT '当前手机使用时长(字典)',
  `address_live_duration` varchar(20)  COMMENT '当前地址居住时长(字典)',
  `credit_card_number` varchar(20)  COMMENT '信用卡数量(字典)',
  `house_status` varchar(20)  COMMENT '住房状态(字典)',
  `other_phone_no` varchar(255)  COMMENT '其他电话号码',
  `email` varchar(128)  COMMENT '邮箱',
  `zalo_id` varchar(255) ,
  `facebook_id` varchar(255) ,
  `inserttime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT '插入时间',
  `updatetime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP COMMENT '更新时间',
  `isactive` tinyint(1) NOT NULL DEFAULT '1' COMMENT '是否逻辑删除(1:不删除)',
  `app_version` varchar(20) ,
  PRIMARY KEY (`id`) USING BTREE,
  UNIQUE KEY `user_id` (`user_id`,`isactive`) USING BTREE
) ENGINE=InnoDB AUTO_INCREMENT=871 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci COMMENT='个人信息';

In [ ]:
%%sql
# 创建用户借款表
CREATE TABLE `loan_list` (
  `id` bigint NOT NULL AUTO_INCREMENT,
  `borrower_id` bigint NOT NULL COMMENT '借款人ID',
  `apply_amount` decimal(18,2) NOT NULL COMMENT '用户申请的额度',
  `period_no` int NOT NULL COMMENT '期数',
  `term_quantity` int NOT NULL COMMENT '每期贷款时长',
  `term_unit` varchar(4) NOT NULL,
  `product_id` int NOT NULL COMMENT '产品ID',
  `prod_type` tinyint NOT NULL COMMENT '1=PDL, 2=INSTALLMENT',
  `interest` decimal(18,2) NOT NULL COMMENT '利息',
  `interest_rate` decimal(10,5) NOT NULL COMMENT '借款利率',
  `service_fee` decimal(18,2) NOT NULL COMMENT 'drools给的应收服务费，实际收取逻辑是service_fee - service_fee_discount',
  `service_fee_discount` decimal(18,2) DEFAULT '0.00' COMMENT '服务费优惠减免',
  `service_rate` decimal(10,5) NOT NULL COMMENT '借款服务费率',
  `service_fee_type` int NOT NULL COMMENT '服务费是否分期1是不分，2是分',
  `overdue_penalty_rate` decimal(10,5) NOT NULL COMMENT '逾期罚息费率',
  `overdue_notify_rate` decimal(10,5) NOT NULL COMMENT '逾期催收费率',
  `overdue_fixed_charge` decimal(10,2) NOT NULL COMMENT '滞纳金',
  `withdraw_adjust_amount` decimal(10,2) DEFAULT NULL COMMENT '提现的时候为了满足尾数需求（假设5的倍数），调整的金额，大于零表示实际提现金额比应提现金额大，小于零表示实际提现金额小于应提现金额',
  `status` int NOT NULL COMMENT '标的状态，兼容历史数据使用',
  `stage` int NOT NULL COMMENT '标的处在的生命周期阶段, 10初始化，30审核，40投标，50转账，70提现，80还款，100结束，负数表示流标',
  `current_stage_status` tinyint NOT NULL COMMENT '当前生命周期的状态， -1失败，0进行中，1成功',
  `list_title` varchar(80) ,
  `list_desc` varchar(200) ,
  `ass_type` int DEFAULT NULL COMMENT '攒标=1, 理财app可投； 不攒标=2,理财app不可投',
  `agreement_id` bigint DEFAULT NULL COMMENT '借款协议id',
  `loan_title` varchar(80) ,
  `loan_purpose_code` varchar(80) ,
  `audit_time` timestamp NULL DEFAULT NULL COMMENT '审核系统给出审核结果时间',
  `full_bid_time` timestamp NULL DEFAULT NULL COMMENT '成标时间',
  `effective_time` timestamp NULL DEFAULT NULL COMMENT '用户收到款项时间',
  `inserttime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT '创建时间',
  `updatetime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP COMMENT '更新时间',
  `isactive` tinyint DEFAULT NULL COMMENT '逻辑删除 null=删除 1=正常',
  `risk_pass_time` timestamp NULL DEFAULT NULL COMMENT '风控通过时间,兼容印尼使用，非印尼请使用audit_time',
  `amount` decimal(18,2) DEFAULT NULL COMMENT '借款金额，额度评估结果',
  `risk_level` varchar(20)  COMMENT 'pata给出标的风险等级',
  `result` text CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
  PRIMARY KEY (`id`) USING BTREE,
  KEY `idx_borrower_id` (`borrower_id`) USING BTREE,
  KEY `idx_status` (`status`) USING BTREE
) ENGINE=InnoDB AUTO_INCREMENT=1107 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci COMMENT='loan list table';

In [ ]:
%%sql
# 创建用户放款表
CREATE TABLE `loan_debt` (
  `id` bigint NOT NULL AUTO_INCREMENT,
  `list_id` bigint NOT NULL COMMENT '标的 id',
  `list_amount` decimal(18,2) NOT NULL COMMENT '标的金额',
  `agreement_id` bigint DEFAULT NULL COMMENT '借款协议id',
  `borrower_id` bigint NOT NULL COMMENT '借款人 id',
  `due_date` timestamp NOT NULL COMMENT '应还时间',
  `period_no` int NOT NULL COMMENT '分期期数',
  `period_seq` int NOT NULL COMMENT '第几期',
  `principal` decimal(18,2) NOT NULL COMMENT '本金',
  `interest` decimal(18,2) NOT NULL COMMENT '利息',
  `service_fee` decimal(18,2) NOT NULL COMMENT '服务费',
  `pre_service_fee` decimal(18,2) DEFAULT '0.00' COMMENT '前置收取的服务费',
  `penalty_fee` decimal(18,2) NOT NULL COMMENT '罚息',
  `overdue_notify_fee` decimal(18,2) NOT NULL COMMENT '催收',
  `amount` decimal(18,2) NOT NULL COMMENT '债务总额',
  `status` tinyint NOT NULL COMMENT 'debt status, 0=正常未还, 1=已还全部, 2=部分还款，3=未还逾期',
  `repay_code_status` tinyint DEFAULT NULL COMMENT '0=未生成, 1=手动生成,2=自动生成,3=生成失败',
  `repay_code_time` timestamp NULL DEFAULT NULL COMMENT '获取还款码的时间',
  `owing_principal` decimal(18,2) NOT NULL COMMENT '未付本金',
  `owing_interest` decimal(18,2) NOT NULL COMMENT '未付利息',
  `owing_service_fee` decimal(18,2) NOT NULL COMMENT '未付服务费',
  `owing_penalty_fee` decimal(18,2) NOT NULL COMMENT '未付罚息',
  `owing_overdue_notify_fee` decimal(18,2) NOT NULL COMMENT '未付罚息',
  `overdue_day` int DEFAULT NULL COMMENT '违约的天数',
  `owing_amount` decimal(18,2) NOT NULL COMMENT '未付债务总额',
  `payment_time` timestamp NULL DEFAULT NULL COMMENT '还款时间',
  `payment_amount` decimal(18,2) DEFAULT NULL COMMENT '已付总额',
  `overdue_fixed_charge` decimal(10,2) NOT NULL COMMENT '滞纳金',
  `user_actual_payment_time` timestamp NULL DEFAULT NULL COMMENT '用户实际还款的时间，来自资金',
  `version` int DEFAULT NULL COMMENT '版本',
  `inserttime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT '创建时间',
  `updatetime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP COMMENT '更新时间',
  `isactive` tinyint DEFAULT NULL COMMENT '逻辑删除 null=删除 1=正常',
  `init_due_date` datetime DEFAULT NULL COMMENT '债务初始化的应还时间',
  `delay_count` int DEFAULT '0' COMMENT '已成功展期的次数',
  PRIMARY KEY (`id`) USING BTREE
) ENGINE=InnoDB AUTO_INCREMENT=7093 DEFAULT CHARSET=utf8mb3 COMMENT='债务';

In [ ]:
%%sql
# 创建用户还款表
CREATE TABLE `tb_repayment_slave_order` (
  `id` bigint NOT NULL AUTO_INCREMENT COMMENT '自增主键',
  `master_order_id` bigint NOT NULL DEFAULT '0' COMMENT '主订单ID',
  `borrower_id` bigint NOT NULL DEFAULT '0' COMMENT '借款人ID',
  `debt_id` bigint NOT NULL DEFAULT '0' COMMENT '债务ID',
  `principal` decimal(18,2) NOT NULL DEFAULT '0.00' COMMENT '还款中的本金',
  `interest` decimal(18,2) NOT NULL DEFAULT '0.00' COMMENT '还款中的利息',
  `penalty_fee` decimal(18,2) NOT NULL DEFAULT '0.00' COMMENT '还款中的罚息',
  `overdue_notify_fee` decimal(18,2) NOT NULL DEFAULT '0.00' COMMENT '还款中的催费',
  `service_fee` decimal(18,2) NOT NULL DEFAULT '0.00' COMMENT '还款中的手续费',
  `inserttime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT '插入时间',
  `updatetime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP COMMENT '更新时间',
  `isactive` tinyint(1) NOT NULL DEFAULT '1' COMMENT '逻辑删除(null=删除,1:未删除)',
  PRIMARY KEY (`id`) USING BTREE
) ENGINE=InnoDB AUTO_INCREMENT=854 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci COMMENT='还款订单明细，可以推导出tb_repayment_record';

In [6]:
%%sql
SELECT * 
FROM u_user ui
    LEFT JOIN u_personal_info pi ON ui.id = pi.user_id
    LEFT JOIN loan_list ll ON ll.borrower_id = ui.id
    LEFT JOIN loan_debt ld ON ld.list_id = ll.id
# LIMIT 100
;

,id,username,mobile,password,nickname,role_type,type,status,on_off,inserttime,...,payment_time,payment_amount,overdue_fixed_charge.1,user_actual_payment_time,version,inserttime.3,updatetime.3,isactive.3,init_due_date,delay_count
0,9999,0258369147,258369147,9d85cd8bf53996df44dddb9a988da293,258369147,1,0,0,1,2020-01-16 19:11:38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10003,7897897897,7897897897,e10adc3949ba59abbe56e057f20f883e,7897897897,1,0,0,1,2019-12-12 19:12:19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10004,01234567896,1234567896,e732f94df609cb4df2a3473a7637e881,1234567896,1,0,0,1,2019-12-12 19:17:23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10005,01234567897,1234567897,9d85cd8bf53996df44dddb9a988da293,1234567897,1,0,0,1,2019-12-12 19:23:41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10006,01234567898,1234567898,9d85cd8bf53996df44dddb9a988da293,1234567898,1,0,0,1,2019-12-12 19:26:48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1687,11046,0855855555,855855555,dc483e80a7a0bd9ef71d8cf973673924,855855555,1,0,0,1,2021-01-07 14:49:04,...,NaN,0.0,30000.0,NaN,2.0,2021-01-11 19:44:18,2021-02-01 12:49:10,1.0,2021-01-25 23:59:59,0.0
1688,11047,0978788425,978788425,9d85cd8bf53996df44dddb9a988da293,978788425,1,0,0,1,2021-02-01 12:33:02,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1689,11047,0978788425,978788425,9d85cd8bf53996df44dddb9a988da293,978788425,1,0,0,1,2021-02-01 12:33:02,...,2021-02-01 13:08:32,748899.0,30000.0,2021-02-01 13:08:31,5.0,2021-02-01 12:40:46,2021-02-01 13:08:32,1.0,2021-02-15 23:59:59,0.0
1690,11048,0123457557,123457557,e732f94df609cb4df2a3473a7637e881,123457557,1,0,0,1,2021-02-01 12:39:45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
%%sql
-- 一个人多次申请, 一个人多条修改记录
-- 我们这里只统计人数, 多条记录也是一个人头
-- 可以使用max 或者 distinct 取出一条用于计算人数

SELECT
    u.id AS user_id, -- 用户ID
    al.id AS list_id,-- 申请ID
    ld.id AS order_id,-- 放款ID
    date( u.inserttime ) AS regist_time,
    max( CASE WHEN pi.user_id IS NOT NULL THEN 1 ELSE 0 END ) AS if_fillin_pi,
    max( CASE WHEN al.borrower_id IS NOT NULL THEN 1 ELSE 0 END ) AS if_apply,
    max( CASE WHEN al.STATUS > 70 THEN 1 ELSE 0 END ) AS if_pass,
    max( CASE WHEN ld.borrower_id IS NOT NULL THEN 1 ELSE 0 END ) AS if_loan,
    max( CASE WHEN ld.payment_amount > 0 THEN 1 ELSE 0 END ) AS if_pay,
    max( CASE WHEN ld.owing_principal = 0 THEN 1 ELSE 0 END ) AS if_pay_1done 
    -- owing 欠款 principal本金
FROM
    u_user AS u
    LEFT JOIN u_personal_info AS pi ON u.id = pi.user_id
    LEFT JOIN loan_list AS al ON al.borrower_id = u.id
    LEFT JOIN loan_debt AS ld ON al.id = ld.list_id 
GROUP BY
    u.id,
    al.id,
    ld.id 
ORDER BY
    u.id

,user_id,list_id,order_id,regist_time,if_fillin_pi,if_apply,if_pass,if_loan,if_pay,if_pay_1done
0,9999,NaN,NaN,2020-01-16,0,0,0,0,0,0
1,10003,NaN,NaN,2019-12-12,0,0,0,0,0,0
2,10004,NaN,NaN,2019-12-12,0,0,0,0,0,0
3,10005,NaN,NaN,2019-12-12,0,0,0,0,0,0
4,10006,NaN,NaN,2019-12-12,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1687,11046,1104.0,7091.0,2021-01-07,1,1,1,1,0,0
1688,11047,1105.0,NaN,2021-02-01,1,1,0,0,0,0
1689,11047,1106.0,7092.0,2021-02-01,1,1,1,1,1,1
1690,11048,NaN,NaN,2021-02-01,0,0,0,0,0,0


In [8]:
%%sql
WITH temp AS (
	SELECT
		u.id AS user_id,
		al.id AS list_id,
		ld.id AS order_id,
		date( u.inserttime ) AS regist_time,
		max( CASE WHEN pi.user_id IS NOT NULL THEN 1 ELSE 0 END ) AS if_fillin_pi,
		max( CASE WHEN al.borrower_id IS NOT NULL THEN 1 ELSE 0 END ) AS if_apply,
		max( CASE WHEN al.STATUS > 70 THEN 1 ELSE 0 END ) AS if_pass,
		max( CASE WHEN ld.borrower_id IS NOT NULL THEN 1 ELSE 0 END ) AS if_loan,
		max( CASE WHEN ld.payment_amount > 0 THEN 1 ELSE 0 END ) AS if_pay,
		max( CASE WHEN ld.owing_principal = 0 THEN 1 ELSE 0 END ) AS if_pay_1done 
	FROM
		u_user AS u
		LEFT JOIN u_personal_info AS pi ON u.id = pi.user_id
		LEFT JOIN loan_list AS al ON al.borrower_id = u.id
		LEFT JOIN loan_debt AS ld ON al.id = ld.list_id 
	GROUP BY
		u.id,
		al.id,
		ld.id 
	ORDER BY
		u.id 
) SELECT
	regist_time,
	count( user_id ) AS regist_num,
	sum( if_fillin_pi ) AS fill_in_pi_num,
	sum( if_apply ) AS apply_num,
	sum( if_pass ) AS pass_num,
	sum( if_loan ) AS loan_num,
	sum( if_pay ) AS pay_num,
	sum( if_apply )/ count( user_id ) AS '注册→申请',
	sum( if_pass )/ sum( if_apply ) AS '申请→通过',
	sum( if_loan )/ sum( if_pass ) AS '通过→放款',
	sum( if_pay )/ sum( if_loan ) AS '放款→还过款',
	sum( if_pay_1done )/ sum( if_loan ) AS '还款→至少1期还完' 
FROM
	temp 
GROUP BY
	regist_time 
ORDER BY
	regist_time

,regist_time,regist_num,fill_in_pi_num,apply_num,pass_num,loan_num,pay_num,注册→申请,申请→通过,通过→放款,放款→还过款,还款→至少1期还完
0,2019-12-12,6,0,0,0,0,0,0.0000,NaN,NaN,NaN,NaN
1,2019-12-13,4,2,0,0,0,0,0.0000,NaN,NaN,NaN,NaN
2,2019-12-16,4,4,2,0,0,0,0.5000,0.0,NaN,NaN,NaN
3,2019-12-17,4,4,0,0,0,0,0.0000,NaN,NaN,NaN,NaN
4,2019-12-18,2,2,0,0,0,0,0.0000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
195,2020-12-11,2,0,0,0,0,0,0.0000,NaN,NaN,NaN,NaN
196,2020-12-14,5,0,0,0,0,0,0.0000,NaN,NaN,NaN,NaN
197,2021-01-07,5,5,5,5,4,3,1.0000,1.0,0.8,0.75,0.75
198,2021-02-01,3,2,2,1,1,1,0.6667,0.5,1.0,1.00,1.00


In [11]:
%%sql
# 获取通过率
select * from loan_list as l

,id,borrower_id,apply_amount,period_no,term_quantity,term_unit,product_id,prod_type,interest,interest_rate,...,audit_time,full_bid_time,effective_time,inserttime,updatetime,isactive,risk_pass_time,amount,risk_level,result
0,1,10013,2000000.0,1,7,D,10000,1,20.0,0.0005,...,2019-12-20 17:24:26,2019-12-30 19:11:28,NaN,2019-12-20 16:36:44,2021-02-22 22:39:33,1,NaN,NaN,NaN,"{""resultsetMap"":{""private_rule_set"":{""ruleSetI..."
1,2,10014,2000000.0,1,7,D,10000,1,7000.0,0.1825,...,2019-12-25 16:41:50,NaN,NaN,2019-12-25 12:52:20,2020-09-16 15:39:17,1,NaN,NaN,NaN,NaN
2,4,10027,1500000.0,1,7,D,10000,1,5250.0,0.1825,...,2019-12-27 16:36:40,2019-12-27 18:31:03,NaN,2019-12-27 16:20:16,2020-07-28 13:38:29,1,NaN,NaN,NaN,NaN
3,5,10016,1500000.0,1,7,D,10000,1,5250.0,0.1825,...,2020-01-10 13:52:53,2020-01-10 13:52:54,NaN,2019-12-27 18:42:47,2020-01-10 15:18:46,1,NaN,NaN,NaN,NaN
4,6,10028,1500000.0,1,14,D,10000,1,10500.0,0.1825,...,2019-12-29 14:02:19,2019-12-29 14:02:20,NaN,2019-12-29 13:11:59,2019-12-29 14:02:22,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,1102,11046,700000.0,1,14,D,10000,1,4900.0,0.1825,...,2021-01-11 15:27:17,2021-01-11 15:27:17,2021-01-11 15:29:58,2021-01-11 15:26:03,2021-01-11 16:53:30,1,2021-01-11 15:27:17,700000.0,NaN,NaN
1085,1103,11046,600000.0,1,14,D,10000,1,4200.0,0.1825,...,2021-01-11 16:56:20,2021-01-11 16:56:21,NaN,2021-01-11 16:55:03,2021-01-11 16:59:55,1,2021-01-11 16:56:20,600000.0,NaN,NaN
1086,1104,11046,700000.0,1,14,D,10000,1,4900.0,0.1825,...,2021-01-11 19:42:04,2021-01-11 19:42:04,2021-01-11 19:44:19,2021-01-11 19:40:43,2021-01-11 19:44:18,1,2021-01-11 19:42:04,700000.0,NaN,NaN
1087,1105,11047,700000.0,1,14,D,10000,1,4900.0,0.1825,...,2021-02-01 12:36:31,NaN,NaN,2021-02-01 12:35:58,2021-02-01 12:36:31,1,2021-02-01 12:36:31,700000.0,NaN,NaN


In [14]:
%%sql
# 定义新客和老客
# 老客：放过款的用户，再来申请就是老客
# 新客：没放过款的客户就是新客，可能是第一次来，之前申请被拒绝了

SELECT 
    borrower_id,
    min(effective_time) AS effective_time
FROM loan_list
WHERE stage IN (80, 100)
GROUP BY borrower_id

,borrower_id,effective_time
0,10034,2020-09-16 16:51:20
1,10040,2020-02-27 23:17:44
2,10043,2019-12-31 09:59:42
3,10044,2019-12-31 13:08:03
4,10045,2019-12-31 19:24:59
...,...,...
264,11029,2020-12-07 16:47:02
265,11031,2020-12-08 14:24:28
266,11037,2020-12-09 10:31:53
267,11046,2021-01-07 16:10:16


In [15]:
%%sql
# 在当前申请时间与第一次成功放款的时间比较，得出新老客户定义
WITH first_loan AS (
    SELECT 
        borrower_id,
        min(effective_time) AS effective_time
    FROM loan_list
    WHERE stage IN (80, 100)
    GROUP BY borrower_id     
) SELECT 
      ll.borrower_id,
      CASE 
        WHEN ll.inserttime > first_loan.effective_time THEN '老客'
        ELSE '新客'
      END AS user_type
FROM loan_list ll
    LEFT JOIN first_loan ON ll.borrower_id = first_loan.borrower_id


,borrower_id,user_type
0,10013,新客
1,10014,新客
2,10027,新客
3,10016,新客
4,10028,新客
...,...,...
1084,11046,老客
1085,11046,老客
1086,11046,老客
1087,11047,新客


In [16]:
%%sql
# 计算整张报表

WITH first_loan AS (
    SELECT 
        borrower_id,
        min(effective_time) AS effective_time
    FROM loan_list
    WHERE stage IN (80, 100)
    GROUP BY borrower_id   
), -- 多个with as同时写的时候，with只能写一次，后面都是 临时表名 + as，select部分需要通过括号包裹起来，多个as需要用都好分割
u_type as (
    SELECT 
        ls.*,
        case when ls.inserttime > first_loan.effective_time THEN '老客' else '新客' end as user_type
    from loan_list ls
        left join first_loan on ls.borrower_id = first_loan.borrower_id
)
SELECT 
    date(l.inserttime) as apply_time,
    user_type,l.period_no,
    l.term_quantity,
    count(l.id) as apply_num,
    sum(case WHEN l.`status`>70 then 1 else 0 end ) as if_pass_num,avg(l.apply_amount) as mean_apply_amount,
    sum(case WHEN l.`status`>70 then 1 else 0 end ) /count(l.id) as passrate -- 申请通过数量/申请数量 计算通过率
from loan_list l 
    LEFT JOIN u_type on u_type.id = l.id
WHERE user_type is not null
 -- 看每天, 不同的客群(新客, 老客) 不同的产品 通过率
GROUP BY apply_time,user_type,l.period_no,l.term_quantity
ORDER BY apply_time,user_type

,apply_time,user_type,period_no,term_quantity,apply_num,if_pass_num,mean_apply_amount,passrate
0,2019-12-20,新客,1,7,1,0,2.000000e+06,0.0
1,2019-12-25,新客,1,7,1,0,2.000000e+06,0.0
2,2019-12-27,新客,1,7,2,0,1.500000e+06,0.0
3,2019-12-29,新客,1,7,1,0,1.500000e+06,0.0
4,2019-12-29,新客,1,14,4,0,1.500000e+06,0.0
...,...,...,...,...,...,...,...,...
319,2020-12-10,老客,1,14,1,1,7.000000e+06,1.0
320,2021-01-07,新客,1,14,1,1,7.000000e+05,1.0
321,2021-01-08,老客,1,14,1,1,7.000000e+05,1.0
322,2021-01-11,老客,1,14,3,3,6.666667e+05,1.0


In [17]:
%%sql
with first_loan as (
    select borrower_id,min(effective_time) as effective_time  
    from loan_list
    where stage in (80,100) -- 成功标志
    group by borrower_id
),
u_type as (
    SELECT 
        ls.*,
        case when ls.inserttime > first_loan.effective_time THEN '老客' else '新客' end as user_type
    from loan_list ls
    left join first_loan on ls.borrower_id = first_loan.borrower_id
)
SELECT 
    date(ld.inserttime) as '放款日',
    user_type,
    l.period_no,
    concat(l.term_quantity,l.term_unit) as '期限', -- concat函数 字符串拼接
    COUNT(DISTINCT list_id) as loan_num,
    sum(ld.list_amount) /COUNT(DISTINCT list_id) as '件均'
FROM loan_debt ld
    LEFT JOIN loan_list l on ld.list_id = l.id
    LEFT JOIN u_type on u_type.id = l.id
WHERE user_type is not null
GROUP BY 1,2,3,4
ORDER BY 1,2

,放款日,user_type,period_no,期限,loan_num,件均
0,2019-12-31,新客,1,7D,2,1500000.0
1,2020-01-03,新客,1,7D,1,3000000.0
2,2020-01-08,新客,1,7D,2,1756000.0
3,2020-01-09,新客,1,7D,3,2000000.0
4,2020-01-10,新客,1,7D,3,1500000.0
...,...,...,...,...,...,...
280,2020-12-10,老客,1,14D,1,7000000.0
281,2021-01-07,新客,1,14D,1,700000.0
282,2021-01-08,老客,1,14D,1,700000.0
283,2021-01-11,老客,1,14D,2,700000.0


In [18]:
%%sql
# vintage报表计算

with cte as (
    SELECT 
        ld.id,date(ld.due_date) as due_date,
        sum(ld.principal) as total_principal,
        sum(case WHEN rl.inserttime is NOT NULL and DATEDIFF(date(rl.inserttime),date(ld.due_date))<=0 THEN rl.principal end) as d0_principal,
        sum(case WHEN rl.inserttime is NOT NULL and DATEDIFF(date(rl.inserttime),date(ld.due_date))=1 THEN rl.principal end) as d1_principal,
        sum(case WHEN rl.inserttime is NOT NULL and DATEDIFF(date(rl.inserttime),date(ld.due_date))=2 THEN rl.principal end) as d2_principal
    from loan_debt ld
    LEFT JOIN tb_repayment_slave_order rl on rl.debt_id = ld.id
    GROUP BY 1, 2
)
select 
    due_date,
    sum(total_principal) as total_principal,
    sum(d0_principal) as d0_principal,
    sum(d1_principal) as d1_principal,
    sum(d2_principal) as d2_principal,
    sum(d0_principal)/sum(total_principal) as d0,
    sum(d1_principal)/sum(total_principal) as d1,
    sum(d2_principal)/sum(total_principal) as d2
from cte
GROUP BY 1

,due_date,total_principal,d0_principal,d1_principal,d2_principal,d0,d1,d2
0,2020-01-07,18000000.0,1500000.0,NaN,NaN,0.083333,NaN,NaN
1,2020-01-10,16500000.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01,30180000.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-11,1500000.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-05,1500000.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
212,2020-12-24,7000000.0,NaN,NaN,NaN,NaN,NaN,NaN
213,2021-01-21,700000.0,700000.0,NaN,NaN,1.000000,NaN,NaN
214,2021-01-10,1400000.0,5000.0,695000.0,NaN,0.003571,0.496429,NaN
215,2021-01-25,1400000.0,700000.0,NaN,NaN,0.500000,NaN,NaN


In [19]:
%%sql
# 催收报表

CREATE TABLE `tb_dun_case_allocation` (
  `id` bigint NOT NULL,
  `borrower_id` bigint NOT NULL COMMENT '借款人ID',
  `allocator_id` bigint NOT NULL COMMENT '分案操作者',
  `status` int NOT NULL COMMENT '1-催收中；2-已被转移走；3-催收结束已关闭',
  `dun_case_id` bigint NOT NULL COMMENT '案件Id',
  `owner_id` bigint NOT NULL COMMENT '经办人',
  `previous_owner_id` bigint DEFAULT NULL COMMENT '前一个经办人',
  `max_overdue_day` int NOT NULL COMMENT '分案时最大逾期天数',
  `inserttime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT '插入时间',
  `updatetime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP COMMENT '更新时间',
  `isactive` tinyint(1) DEFAULT NULL COMMENT '逻辑删除(null:删除,1:未删除)',
  PRIMARY KEY (`id`) USING BTREE,
  KEY `ix_borrower_id` (`borrower_id`) USING BTREE,
  KEY `ix_dun_case_id` (`dun_case_id`) USING BTREE,
  KEY `ix_owner_id` (`owner_id`) USING BTREE,
  KEY `ix_status` (`status`) USING BTREE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb3 COMMENT='催收分案';


Exception: [42S01][1050] Table 'tb_dun_case_allocation' already exists

In [20]:
%%sql
# 催收分案明细
CREATE TABLE `tb_dun_case_allocation_detail` (
  `id` bigint NOT NULL AUTO_INCREMENT COMMENT '自增主键',
  `borrower_id` bigint NOT NULL COMMENT '借款人ID',
  `allocation_id` bigint NOT NULL COMMENT '分案Id',
  `dun_case_id` bigint NOT NULL COMMENT '案件Id',
  `list_id` bigint NOT NULL COMMENT '标的 id',
  `debt_id` bigint NOT NULL COMMENT '债务ID',
  `owner_id` bigint NOT NULL COMMENT '经办人',
  `overdue_day` int NOT NULL COMMENT '分案时逾期天数',
  `start_owing_amount` decimal(18,2) NOT NULL COMMENT '起始逾期金额',
  `start_owing_principal` decimal(18,2) NOT NULL COMMENT '起始逾期本金',
  `start_owing_interest` decimal(18,2) NOT NULL COMMENT '起始逾期利息',
  `start_owing_notify_fee` decimal(18,2) NOT NULL COMMENT '起始逾期公司罚息',
  `start_owing_penalty_fee` decimal(18,2) NOT NULL COMMENT '起始逾期用户罚息',
  `start_owing_service_fee` decimal(18,2) NOT NULL COMMENT '起始逾期用户服务费',
  `dun_over_time` timestamp NULL DEFAULT NULL COMMENT '结束时间，还清',
  `dun_repay_amount` decimal(18,2) NOT NULL COMMENT '已还金额',
  `dun_repay_principal` decimal(18,2) NOT NULL COMMENT '已还本金',
  `dun_repay_interest` decimal(18,2) NOT NULL COMMENT '已还利息',
  `dun_repay_notify_fee` decimal(18,2) NOT NULL COMMENT '已还公司罚息',
  `dun_repay_penalty_fee` decimal(18,2) NOT NULL COMMENT '已还用户罚息',
  `dun_repay_service_fee` decimal(18,2) NOT NULL COMMENT '已还服务费',
  `dun_repay_status` int NOT NULL COMMENT '入催后的还款状态，跟随loan_debt，2-已全全部，3-部分还款，4-未还逾期',
  `inserttime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT '插入时间',
  `updatetime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP COMMENT '更新时间',
  `isactive` tinyint(1) DEFAULT NULL COMMENT '逻辑删除(null:删除,1:未删除)',
  `delay_flag` tinyint(1) DEFAULT '0' COMMENT '对应债务在催期间是否展期',
  PRIMARY KEY (`id`) USING BTREE,
  KEY `ix_allocation_id` (`allocation_id`) USING BTREE,
  KEY `ix_borrower_id` (`borrower_id`) USING BTREE,
  KEY `ix_debt_id` (`debt_id`) USING BTREE,
  KEY `ix_dun_case_id` (`dun_case_id`) USING BTREE
) ENGINE=InnoDB AUTO_INCREMENT=129 DEFAULT CHARSET=utf8mb3 COMMENT='催收分案明细';

Exception: [42S01][1050] Table 'tb_dun_case_allocation_detail' already exists

In [21]:
%%sql
# 催收人员表
CREATE TABLE `tb_backend_user` (
  `id` bigint NOT NULL AUTO_INCREMENT COMMENT '自增主键',
  `username` varchar(64)  NOT NULL,
  `realname` varchar(128)  NOT NULL COMMENT '用户真实姓名',
  `mobile` varchar(20)  DEFAULT NULL,
  `password` varchar(64)  DEFAULT NULL,
  `email` varchar(64)  DEFAULT NULL,
  `email_password` varchar(64)  DEFAULT '' COMMENT '用户邮箱密码',
  `department` varchar(128)  DEFAULT NULL COMMENT '部门',
  `department_id` bigint NOT NULL,
  `role_type` int DEFAULT NULL COMMENT '角色(-1:普通用户)',
  `type` int DEFAULT NULL COMMENT '0：借款用户,1:资金账户用户',
  `status` int NOT NULL COMMENT '用户状态',
  `on_off` char(4)  NOT NULL COMMENT '开关(0:关闭,1:开启)',
  `operate_id` bigint NOT NULL DEFAULT '0' COMMENT '操作人的id',
  `softphone_account` varchar(20)  DEFAULT NULL COMMENT '外呼系统软电话坐席号/分机号',
  `inserttime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT '插入时间',
  `updatetime` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
  `isactive` tinyint(1) DEFAULT NULL COMMENT '逻辑删除',
  PRIMARY KEY (`id`) USING BTREE,
  UNIQUE KEY `uk_username` (`username`,`isactive`) USING BTREE
) ENGINE=InnoDB AUTO_INCREMENT=84 DEFAULT CHARSET=utf8mb3 COMMENT='后台用户表';

Exception: [42S01][1050] Table 'tb_backend_user' already exists

In [23]:
%%sql
# 定义分案归属, 当天最后一次分案算作最终分案归属
SELECT max(id) as id from tb_dun_case_allocation GROUP BY date(inserttime), dun_case_id

,id
0,89
1,95
2,100
3,131
4,262
...,...
104,3769
105,3779
106,3988
107,4028


In [26]:
%%sql
# 分案表, 计算逾期天数还需要loan_debt
with cte1 as (
    SELECT max(id) as id 
    from tb_dun_case_allocation 
    GROUP BY date(inserttime), dun_case_id
),
cte2 as (
    select
        cte1.id,
        dca.borrower_id,
        dca.dun_case_id,
        dcad.list_id,
        dcad.debt_id,
        dcad.inserttime as fenan_time, 
        dcad.overdue_day,
        ld.due_date,
        dcad.owner_id,
        dcad.start_owing_amount,
        dcad.start_owing_principal,
        dcad.dun_repay_amount,
        dcad.dun_repay_principal
from cte1
    LEFT JOIN tb_dun_case_allocation dca on cte1.id = dca.id
    LEFT JOIN tb_dun_case_allocation_detail dcad on dcad.allocation_id = dca.id 
    LEFT JOIN loan_debt ld on ld.id = dcad.debt_id
WHERE date(dcad.inserttime)>=date('2020-01-07') 
  and dcad.overdue_day<=7 ORDER BY dcad.debt_id
),
c as (
    SELECT
        date(cte2.fenan_time) as fenan_time,
        bu.realname,sum(cte2.start_owing_amount) as lj_owing_amount,
        sum(cte2.dun_repay_amount) as lj_repay_amount,
        sum(cte2.dun_repay_amount)/sum(cte2.start_owing_amount) as repay_rate_amount, sum(cte2.start_owing_principal) as lj_owing_principal,
        sum(cte2.dun_repay_principal) as lj_repay_principal,sum(cte2.dun_repay_principal)/sum(cte2.start_owing_principal) as repay_rate_principal,
        count(cte2.dun_case_id) as fenan_cnt,sum(case WHEN cte2.dun_repay_amount>0 then 1 else 0 end) as repay_cnt,
        SUM(case WHEN cte2.dun_repay_amount>0 then 1 else 0 end)/COUNT(cte2.dun_case_id) as repay_rate_cnt
    FROM cte2 
        left JOIN tb_backend_user bu on cte2.owner_id = bu.id
    WHERE cte2.start_owing_amount>=10000 and bu.realname in ('test','test2','test1','test01')
    GROUP BY 1,2
    ORDER BY 1,2
)
SELECT 
    c.fenan_time as '分案日',
    c.realname as '姓名',
    0.38 AS '目标',
    c.lj_owing_amount as '分案金额',
    c.lj_repay_amount  as ' 还款金额',
    c.repay_rate_amount as '还款率-金额',
    c.lj_owing_principal as '分案本金' ,
    c.lj_repay_principal as '还款本金' ,
    c.repay_rate_principal as '还款率-本金',
    c.fenan_cnt as '分案件数',
    c.repay_cnt as '还款件数',
    c.repay_rate_cnt as '还款率-件数'
from c                                                         

,分案日,姓名,目标,分案金额,还款金额,还款率-金额,分案本金,还款本金,还款率-本金,分案件数,还款件数,还款率-件数
0,2020-03-23,test01,0.38,1535250.0,1535250.0,1.000000,1500000.0,1500000.0,1.000,1,1,1.0
1,2020-03-26,test01,0.38,3075750.0,3075750.0,1.000000,3000000.0,3000000.0,1.000,2,2,1.0
2,2020-03-31,test01,0.38,1088500.0,1088500.0,1.000000,1000000.0,1000000.0,1.000,1,1,1.0
3,2020-04-01,test01,0.38,1023500.0,1993842.0,1.948063,1000000.0,1000000.0,1.000,1,1,1.0
4,2020-04-02,test01,0.38,1088500.0,1088500.0,1.000000,1000000.0,1000000.0,1.000,1,1,1.0
5,2020-04-07,test01,0.38,1128500.0,53000.0,0.046965,1000000.0,53000.0,0.053,1,1,1.0
6,2020-04-09,test01,0.38,801535.0,801535.0,1.000000,800000.0,800000.0,1.000,1,1,1.0
7,2020-04-24,test01,0.38,4395841.0,4395841.0,1.000000,4248776.0,4248776.0,1.000,4,4,1.0
8,2020-04-29,test01,0.38,1255102.0,1255102.0,1.000000,1200000.0,1200000.0,1.000,1,1,1.0
9,2020-05-08,test01,0.38,2761659.0,2761659.0,1.000000,2700000.0,2700000.0,1.000,3,3,1.0
